| [**Overview**](./00_overview.ipynb) | [Getting Started](./01_jupyter_python.ipynb) | **Examples:** | [Access](./02_accessing_indexing.ipynb) | [Transform](./03_transform.ipynb) | [Plotting](./04_simple_vis.ipynb) | [Norm-Spiders](./05_norm_spiders.ipynb) | [Minerals](./06_minerals.ipynb) | **Workflows:** | [lambdas](./07_lambdas.ipynb) | [CIPW](./08_CIPW_Norm.ipynb)  | [ML](./11_geochem_ML.ipynb) | [Spatial Data](./12_spatial_geochem.ipynb) |
| -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- |

# Simple Visualisation:<br> Bivariate, Ternary and Density Plots

`pyrolite` contains an array of visualisation methods, a few of which we'll quickly run through here. For more, check out the [examples gallery](https://pyrolite.readthedocs.io/en/develop/examples/index.html#plotting-examples)!

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from synthdata import get_synthetic_data

In [ ]:
# here we specify a covariance matrix - this simply tells it 'how spread out' we want the data

df = get_synthetic_data(
    columns=["SiO2", "CaO", "MgO", "FeO", "TiO2"],
    cov=np.eye(4) * np.array([0.35, 0.85, 0.3, 1.1]),
    size=1000,
    seed=13,
)


---
## Bivariate Plotting

While there are many ways to get to simple bivariate plots, `pyrolite` provides a few options which can provide a simpler interface and easier access to simple styling configuration.

In [ ]:
df[["MgO", "SiO2"]].pyroplot.scatter(color="k", marker="o", alpha=0.5)

Where we get to larger datasets, overplotting becomes an issue, and we may want to consider methods for visualising the distribution of data as a whole rather than individual points. `pyrolite` has as few options for this, including 'density' plots and 'heatscatter' plots (based on kernel density estimates).

In [ ]:
df[["MgO", "SiO2"]].pyroplot.density(bins=100)

While this can look quite nice, and it solves the issue of overplotting we were nearing above, sometimes we want to be able to plot over this and clearly see where new data sits. In this case, we can instead use percentile contours of the kernel density estimate instead:

In [ ]:
df[["MgO", "SiO2"]].pyroplot.density(
    bins=100, contours=[0.5, 0.95], colors=["k", "0.5"]
)

While it does look nice, the data density diagram and contours both show kernel density which crosses the axes - which shouldn't occur when we consider that abundances are positive-only! One way to get around this is to use a log-scaled kernel density grid. We can see that this improves the situation for both figures:

Note also that we can change colormaps - matplotlib has [a decent range to choose from](https://matplotlib.org/stable/tutorials/colors/colormaps.html), noting that you should lean towards linear unidirectional colormaps in most geochemical data cases!

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 4))
df[["MgO", "SiO2"]].pyroplot.density(
    ax=ax[0], bins=100, logx=True, logy=True, cmap="Purples"
)
df[["MgO", "SiO2"]].pyroplot.density(
    ax=ax[1], bins=100, contours=[0.5, 0.95], colors=["k", "0.5"], logx=True, logy=True
)

Another way to visualise your data is to combine summary information provided by the kernel density estimate with the scatter plot to produce a heatscatter plot which retains the best of both worlds. Here we can see the position of each invdividual sample beyond the core of the distribution, and also itentify where the greatest density of samples are:

In [ ]:
df[["MgO", "SiO2"]].pyroplot.heatscatter(alpha=0.5, logx=True, logy=True)

---
## Ternary Plots

Ternary plots are a common in geochemistry, mineralogy and petrology but dont' necessarily pop up elsewhere. `pyrolite` provides an interface to create ternary plots wherever you pass three columns, making it as simple as creating our bivariate plots above!

In [ ]:
df[["CaO", "MgO", "FeO"]].pyroplot.scatter(color="k", marker="o", alpha=0.5)

In contrast to most ternary plots, however, we can also create data density visualisations (based on distributions in logratio space):

In [ ]:
df[["CaO", "MgO", "FeO"]].pyroplot.density(contours=[0.5, 0.95], colors=["k", "0.5"])

In [ ]:
df[["CaO", "MgO", "FeO"]].pyroplot.heatscatter(alpha=0.5, cmap="cividis", s=3)

---
## Plot Templates

pyrolite provides a few built-in plot templates, with the idea to expand the collection in the near future. 

In [ ]:
from pyrolite.plot.templates import pearceThNbYb, TAS

In [ ]:
ax = pearceThNbYb()

In [ ]:
ax = TAS()

This TAS diagram is a litte more than it seems - it's actually built upon a classifer, which will allow you to automatically classify your samples if you have the relevant data:

In [ ]:
from pyrolite.util.classification import TAS
from synthdata import get_synthetic_data

df = get_synthetic_data(
    columns=["SiO2", "Na2O", "K2O", "Al2O3"],
    mean=[0.5, 0.04, 0.05, 0.4],
    size=100,
    seed=49,
)
df["Na2O + K2O"] = df["Na2O"] + df["K2O"]

cm = TAS()  # TAS classifier model
df["TAS"] = cm.predict(df)  # predict what TAS class the samples occupy

In [ ]:
ax = cm.add_to_axes(
    alpha=0.5, linewidth=0.5, zorder=-1, labels="ID"
)  # add the TAS diagram with the labels to an axis
df[["SiO2", "Na2O + K2O"]].pyroplot.scatter(
    ax=ax, c=df["TAS"], alpha=0.5
)  # plot the data colored by their TAS class

## A Quick Aside: Exporting Figures

We quickly mentioned exporting tables from `pandas` earlier (see the [range of import and export options](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html)), and here we can have a look at the options exporting for `matplotlib` figures. This is largely centered around the [`save_figure` function](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.savefig.html), and `pyrolite` provides a bit of a shortcut to this if you'd like to use it.

A range of export formats are available - in standard raster (JPEG, PNG, TIFF etc) and vector formats (PDF, SVG, EPS etc).

In [ ]:
from pyrolite.util.plot import save_figure

save_figure(ax.figure, name="TAS_Diagram")  # defualt PNG output

The `save_figure` function also allows you to export in multiple formats at the same time:

In [ ]:
save_figure(ax.figure, name="TAS_Diagram.pdf", save_fmts=["png", "pdf"], dpi=300)

---

Check out some of the other templates in `pyrolite.plot.templates` (e.g. the `USDASoilTexture` Triangle).

In [ ]:
import pyrolite.plot.templates

diagrams = [
    d
    for d in dir(pyrolite.plot.templates)
    if (d[0].upper() == d[0]) and not d.startswith("_")
]
diagrams


----
<div class='alert alert-warning'> <font size="+1" color="black"><b> Checkpoint & Time Check</b><br>How are things going?</font></div>

----

| [**Overview**](./00_overview.ipynb) | [Getting Started](./01_jupyter_python.ipynb) | **Examples:** | [Access](./02_accessing_indexing.ipynb) | [Transform](./03_transform.ipynb) | [Plotting](./04_simple_vis.ipynb) | [Norm-Spiders](./05_norm_spiders.ipynb) | [Minerals](./06_minerals.ipynb) | **Workflows:** | [lambdas](./07_lambdas.ipynb) | [CIPW](./08_CIPW_Norm.ipynb)  | [ML](./11_geochem_ML.ipynb) | [Spatial Data](./12_spatial_geochem.ipynb) |
| -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- |